# given sample text, predict similar job postings

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess

from nltk.corpus import stopwords
STOPWORDS = stopwords.words('english')

import pickle

In [2]:
DATA_PATH = '/Users/richardkuzma/coding/analysis/monster/data/cleaned/'

with open(DATA_PATH + 'monster_jobs_df_small.pkl', 'rb') as f:
    df = pickle.load(f)


In [3]:
df.head()

,id,job_title,job_description,job_type,location,organization,sector
0,0,IT Support Technician Job in Madison,TeamSoft is seeing an IT Support Specialist to...,Full Time Employee,"Madison, WI 53702",None,IT/Software Development
1,1,Business Reporter/Editor Job in Madison,The Wisconsin State Journal is seeking a flexi...,Full Time,"Madison, WI 53708",Printing and Publishing,None
2,2,Johnson & Johnson Family of Companies Job Appl...,Report this job About the Job DePuy Synthes Co...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,None
3,3,Engineer - Quality Job in Dixon,Why Join Altec? If you’re considering a career...,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager)
4,4,Shift Supervisor - Part-Time Job in Camphill,Position ID# 76162 # Positions 1 State CT C...,Full Time Employee,"Camphill, PA",Retail,Project/Program Management


In [4]:
DATA_PATH = '/Users/richardkuzma/coding/analysis/monster/data/cleaned/'

with open(DATA_PATH + 'monster_jobs_cleaned_text.pkl', 'rb') as f:
    cleaned_text = pickle.load(f)

In [5]:
df['cleaned_description'] = cleaned_text

In [6]:
df.head()

,id,job_title,job_description,job_type,location,organization,sector,cleaned_description
0,0,IT Support Technician Job in Madison,TeamSoft is seeing an IT Support Specialist to...,Full Time Employee,"Madison, WI 53702",None,IT/Software Development,"[teamsoft, see, support, specialist, join, cli..."
1,1,Business Reporter/Editor Job in Madison,The Wisconsin State Journal is seeking a flexi...,Full Time,"Madison, WI 53708",Printing and Publishing,None,"[wisconsin, state, journal, seek, flexible, mo..."
2,2,Johnson & Johnson Family of Companies Job Appl...,Report this job About the Job DePuy Synthes Co...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,None,"[report, job, job, depuy_synthes, company, mem..."
3,3,Engineer - Quality Job in Dixon,Why Join Altec? If you’re considering a career...,Full Time,"Dixon, CA",Altec Industries,Experienced (Non-Manager),"[join, altec, consider, career, altec, inc, ne..."
4,4,Shift Supervisor - Part-Time Job in Camphill,Position ID# 76162 # Positions 1 State CT C...,Full Time Employee,"Camphill, PA",Retail,Project/Program Management,"[position, id_positions, state, ct, city, fair..."


In [7]:
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])

docs = list(df['cleaned_description'])
labels = list(df['id'])

sentences = TaggedDocumentIterator(docs, labels)

In [11]:
# train Doc2Vec model  on jobs
model = Doc2Vec(vector_size=100,
                 window=5,
                 min_count=20,
                 workers=1,
                 alpha=0.025,
#                  min_alpha=0.0025,
                 epochs=50,
                 dm=0,        #1 = paragraph vector - distributed memory; 0 = dbow
                 seed=42)

model.build_vocab(sentences)
model.train(sentences,
             total_examples=model.corpus_count,
             epochs=model.epochs)

print('created Doc2Vec model')
print('Distributed Mem (True) or BOW (false): {}'.format(model.dm))
print('vector length: {}'.format(model.vector_size))
print('corpus count: {}'.format(model.corpus_count))
print('epochs: {}'.format(model.epochs))



created Doc2Vec model
Distributed Mem (True) or BOW (false): False
vector length: 100
corpus count: 22000
epochs: 50


<em> vector length ~100 works better than 1000? </em> <br>
<em> getting rid of min_alpha helps </em>

In [12]:
#save re-load model
MODEL_PATH = '/Users/richardkuzma/coding/analysis/monster/models/'
model_name = 'd2v_' + 'dm_' + str(int(model.dm)) +'_size_' + str(model.vector_size) + '_epochs_' + str(model.epochs) + '.doc2vec'
model.save(MODEL_PATH + model_name)



# Load Model

In [13]:
model = Doc2Vec.load(MODEL_PATH + model_name)


In [14]:
def print_job_info(num):

    print('Job id: {}'.format(df.iloc[num]['id']))
    print('Job title: {}'.format(df.iloc[num]['job_title']))
    print('Company: {}'.format(df.iloc[num]['organization']))
    print('Job type: {}'.format(df.iloc[num]['job_type']))
    print('Sector: {}'.format(df.iloc[num]['sector']))
    print('Location: {}'.format(df.iloc[num]['location']))

    print('\nDescription: \n{}'.format(df.iloc[num]['job_description']) + '\n'*3)

In [15]:
# sample job is the 0th job posting
sample1 = df.iloc[0]['job_description']
sample2 = df.iloc[15]['job_description']
sample3 = df.iloc[18]['job_description']
    

In [26]:
def predict_jobs(model, df, text, topn=20):
    print("\nSearching for matches for the following document: \n{}".format(text))
    
    #Convert the sample document into a list and use the infer_vector method to get a vector representation for it
    pick_words = [word for word in simple_preprocess(str(text), deacc=True) if word not in STOPWORDS]
#     pick_words = text.split()
    pick_vec = model.infer_vector(pick_words, epochs=100, alpha=0.025)
 
    #use the most_similar utility to find the most similar documents.
    similars = model.docvecs.most_similar(positive=[pick_vec], topn=topn)
          
    print('\n'*4)
    print('-'*80)
    print('\t'*4 + 'Preprocessed text')
    print('-'*80)
    print(pick_words)
    
    
    print('\n'*4)
    print('-'*80)
    print('\t'*4 + 'Similar job postings')
    print('-'*80)
    
    count = 1
    for i in similars:
        print('\n\n\nSimilar job number: {}'.format(count))
        count += 1
        print('Job ID ', i[0], ' Similarity Score: ', i[1])
        print('~'*50 + '\n')

        print_job_info(i[0])
        
        


### prediction

In [22]:
predict_jobs(model=model, df=df, text=sample1, topn=20)


Searching for matches for the following document: 
TeamSoft is seeing an IT Support Specialist to join our client in Madison, WI. The ideal candidate must have at least 6 years of experience in the field. They need to be familiar with a variety of the field's concepts, practices, and procedures as this position relies on extensive experience and judgment to plan and accomplish goals. Required Skills:• Call tracking software• Phone based technical support• Problem documentation and communication• Remote Desktop Management Tools• Respond to customer requests• General understanding of LANDesk• Microsoft Office 2007 SuiteFind out why TeamSoft is the Madison area's technology leader with over 200 IT consultants. Owned, operated, and managed by IT consultants, TeamSoft is the clear choice for your career. Join the TeamSoft family today!Did you know?• TeamSoft offers salaried options for many positions that include 5 weeks PTO and great benefits!• TeamSoft has been in business since 1996• Te

In [24]:
predict_jobs(model=model, df=df, text=sample2, topn=20)


Searching for matches for the following document: 
CenturyLink (NYSE: CTL) is a global communications, hosting, cloud and IT services company enabling millions of customers to transform their businesses and their lives through innovative technology solutions. CenturyLink offers network and data systems management, Big Data analytics and IT consulting, and operates more than 55 data centers in North America, Europe and Asia. The company provides broadband, voice, video, data and managed services over a robust 250,000-route-mile U.S. fiber network and a 300,000-route-mile international transport network.Job Responsibilties :The Engineer II responsibilities include design and preparation of Engineering Work Orders for all growth whether Land developments, new buildings, single line requests, Government & Safety, Preservation, Rehab, DSL, FTTN, FTTH, FTTT, Road Relocation, augmentation projects and circuit/special circuit requests. Utilizing the following tools; Bidmaster/Fireworks, JDS, 

In [25]:
predict_jobs(model=model, df=df, text=sample3, topn=20)


Searching for matches for the following document: 
Job SummaryDo you have a strong aptitude for writing, interest in learning new technologies, and a flair for explaining complex concepts and procedures to technical audiences? You might be: Experienced in MATLAB or C/C++ programming A recent graduate in engineering or computer science with a keen interest in technical communication An experienced writer with a strong technical background Come join a team at MathWorks that writes online content to explain how to use sophisticated products that automatically generate C/C++ code from MATLAB algorithms. Code generation technology is revolutionizing engineering. Software components make up an ever larger part of everyday devices (cars, planes, cell phones, and more).  Engineers use tools like ours to model systems at a high level and then turn those models into embeddable C code for real-time and embedded systems.ResponsibilitiesAt MathWorks, documentation is a key component of our product

### Repeat jobs in dataset mean you see the same job multiple times...
- ...though they are 'similar' in how they're written

#### Make some fake job descriptions...
pulled these off of <a href=https://hiring.monster.com/employer-resources/job-description-templates/page/4/> monster.com job description templates </a>

- business analyst
- senior software eng
- registered nurse (anesthetist)
- data engineer


In [37]:
example_biz_analyst = "Business Analyst Job Responsibilities:  Elicits, analyzes, specifies, and validates the business needs of stakeholders, be they customers or end users. Collaborates with project sponsors to determine project scope and vision. Clearly identifies project stakeholders and establish customer classes, as well as their characteristics. Conducts interviews to gather customer requirements via workshops, questionnaires, surveys, site visits, workflow storyboards, use cases, scenarios, and other methods. Identifies and establishes scope and parameters of requirements analysis on a project-by-project basis to define project impact, outcome criteria, and metrics. Works with stakeholders and project team to prioritize collected requirements. Researches, reviews, and analyzes the effectiveness and efficiency of existing requirements-gathering processes and develop strategies for enhancing or further leveraging these processes. Assists in conducting research on products to meet agreed upon requirements and to support purchasing efforts. Participates in the QA of purchased solutions to ensure features and functions have been enabled and optimized. Participates in the selection of any requirements documentation software solutions that the organization may opt to use. Analyzes and verifies requirements for completeness, consistency, comprehensibility, feasibility, and conformity to standards. Develops and utilizes standard templates to accurately and concisely write requirements specifications. Translates conceptual customer requirements into functional requirements in a clear manner that is comprehensible to developers/project team. Creates process models, specifications, diagrams, and charts to provide direction to developers and/or the project team. Develops and conduct peer reviews of the business requirements to ensure that requirement specifications are correctly interpreted. Assists with the interpretation of customer requirements into feasible options, and communicating these back to the business stakeholders. Manages and tracks the status of requirements throughout the project lifecycle; enforce and redefine as necessary. Communicates changes, enhancements, and modifications of business requirements — verbally or through written documentation — to project managers, sponsors, and other stakeholders so that issues and solutions are understood. Provides guidance and/or instruction to junior staff members.  "
example_software_eng = "Senior Software Engineer Job Duties: Develops software solutions by studying information needs; conferring with users; studying systems flow, data usage, and work processes; investigating problem areas; following the software development lifecycle. Determines operational feasibility by evaluating analysis, problem definition, requirements, solution development, and proposed solutions. Documents and demonstrates solutions by developing documentation, flowcharts, layouts, diagrams, charts, code comments and clear code. Prepares and installs solutions by determining and designing system specifications, standards, and programming. Improves operations by conducting systems analysis; recommending changes in policies and procedures. Updates job knowledge by studying state-of-the-art development tools, programming techniques, and computing equipment; participating in educational opportunities; reading professional publications; maintaining personal networks; participating in professional organizations. Protects operations by keeping information confidential. Provides information by collecting, analyzing, and summarizing development and service issues. Accomplishes engineering and organization mission by completing related results as needed. Supports and develops software engineers by providing advice, coaching and educational opportunities. Skills/Qualifications: Analyzing Information, Software Design, Software Documentation, Software Testing, Teamwork, General Programming Skills, Software Development Fundamentals, Software Development Process, Software Requirements, Software Architecture, Coaching" 
example_registered_nurse = "Certified Registered Nurse Anesthetist Job Responsibilities:  Provides pre-anesthetic preparation and patient evaluation. Ensures patient identification and obtains appropriate health history. Recommends, requests, and evaluates pertinent diagnostic studies. Documents pre-anesthetic evaluation. Obtains informed consent for anesthesia. Selects and/or administers pre-anesthetic medications. Selects prepares and administers anesthetic agents or other agents administered in management of anesthetic care. Informs anesthesiologist and/or surgeon of changes in patient’s condition. Provides anesthesia induction, maintenance, emergence, and post anesthesia care. Inserts invasive line catheter/devices. Performs tracheal intubation and extubation, airway management. Provides mechanical ventilation. Performs venous and arterial punctures. Obtains blood samples. Performs and manages regional anesthetic. Manages patient’s fluid, blood, electrolyte and acid base balance. Provides perianesthetic invasive and non-invasive monitoring utilizing current standards and techniques. Responds to abnormal findings with corrective action. Recognizes and treats cardiac dysrhythmias through use of perianesthetic electrocardiogram monitoring. Evaluates patient response during emergence from anesthesia. Institutes pharmacological or supportive treatment to insure adequacy of patient recovery from anesthesia and adjuvant drugs. Provides post anesthesia follow-up, report, and evaluation of patient’s response to anesthesia and for potential anesthetic complication. Identifies and manages emergency situations. Initiates or participates in cardiopulmonary resuscitation. Performs or orders equipment safety checks as needed. Cleans and sterilizes equipment and notifies supervisor of needed equipment adjustments/repairs. May perform patient care to the extent necessary to maintain clinical expertise, competency and licensing necessary to fulfill job responsibilities and to direct the provision of care on the unit. Education, Experience, and Licensing Requirements:  Graduate of accredited nurse anesthesia program To (2) years of anesthesia care in acute setting experience preferred Valid state RN License (must meet education requirement(s) for state licensure) Valid state APRN Recognition (must meet education requirement (s) for state recognition and obtain within six (6) months of hire) Certified Registered Nurse Anesthetist (CRNA) by the American Association of Nurse Anesthetists (AANA) Certified Advanced Cardiac Life Support (ACLS) by the American Heart Association Current BLS for Healthcare Provider CPR or CPR/AED for the Professional Rescuer certification National Provider Identifier (NPI) and Taxonomy code required at time of hire"  
example_data_engineer = "Data Engineer Job Responsibilities:  Develops and maintains scalable data pipelines and builds out new API integrations to support continuing increases in data volume and complexity. Collaborates with analytics and business teams to improve data models that feed business intelligence tools, increasing data accessibility and fostering data-driven decision making across the organization. Implements processes and systems to monitor data quality, ensuring production data is always accurate and available for key stakeholders and business processes that depend on it. Writes unit/integration tests, contributes to engineering wiki, and documents work. Performs data analysis required to troubleshoot data related issues and assist in the resolution of data issues. Works closely with a team of frontend and backend engineers, product managers, and analysts. Defines company data assets (data models), spark, sparkSQL, and hiveSQL jobs to populate data models. Designs data integrations and data quality framework. Designs and evaluates open source and vendor tools for data lineage. Works closely with all business units and engineering teams to develop strategy for long term data platform architecture.  Data Engineer Qualifications / Skills:  Knowledge of best practices and IT operations in an always-up, always-available service Experience with or knowledge of Agile Software Development methodologies Excellent problem solving and troubleshooting skills Process oriented with great documentation skills Excellent oral and written communication skills with a keen sense of customer service Education, Experience, and Licensing Requirements:  BS or MS degree in Computer Science or a related technical field 4+ years of Python or Java development experience 4+ years of SQL experience (No-SQL experience is a plus) 4+ years of experience with schema design and dimensional data modeling Ability in managing and communicating data warehouse plans to internal clients E perience designing, building, and maintaining data processing systems Experience working with either a Map Reduce or an MPP system on any size/scale"


In [33]:
predict_jobs(model=model, df=df, text=example_biz_analyst, topn=20)


Searching for matches for the following document: 
Business Analyst Job Responsibilities:  Elicits, analyzes, specifies, and validates the business needs of stakeholders, be they customers or end users. Collaborates with project sponsors to determine project scope and vision. Clearly identifies project stakeholders and establish customer classes, as well as their characteristics. Conducts interviews to gather customer requirements via workshops, questionnaires, surveys, site visits, workflow storyboards, use cases, scenarios, and other methods. Identifies and establishes scope and parameters of requirements analysis on a project-by-project basis to define project impact, outcome criteria, and metrics. Works with stakeholders and project team to prioritize collected requirements. Researches, reviews, and analyzes the effectiveness and efficiency of existing requirements-gathering processes and develop strategies for enhancing or further leveraging these processes. Assists in conducting

In [34]:
predict_jobs(model=model, df=df, text=example_software_eng, topn=20)


Searching for matches for the following document: 
Senior Software Engineer Job Duties: Develops software solutions by studying information needs; conferring with users; studying systems flow, data usage, and work processes; investigating problem areas; following the software development lifecycle. Determines operational feasibility by evaluating analysis, problem definition, requirements, solution development, and proposed solutions. Documents and demonstrates solutions by developing documentation, flowcharts, layouts, diagrams, charts, code comments and clear code. Prepares and installs solutions by determining and designing system specifications, standards, and programming. Improves operations by conducting systems analysis; recommending changes in policies and procedures. Updates job knowledge by studying state-of-the-art development tools, programming techniques, and computing equipment; participating in educational opportunities; reading professional publications; maintaining pe

In [40]:
predict_jobs(model=model, df=df, text=example_registered_nurse, topn=20)


Searching for matches for the following document: 
Certified Registered Nurse Anesthetist Job Responsibilities:  Provides pre-anesthetic preparation and patient evaluation. Ensures patient identification and obtains appropriate health history. Recommends, requests, and evaluates pertinent diagnostic studies. Documents pre-anesthetic evaluation. Obtains informed consent for anesthesia. Selects and/or administers pre-anesthetic medications. Selects prepares and administers anesthetic agents or other agents administered in management of anesthetic care. Informs anesthesiologist and/or surgeon of changes in patient’s condition. Provides anesthesia induction, maintenance, emergence, and post anesthesia care. Inserts invasive line catheter/devices. Performs tracheal intubation and extubation, airway management. Provides mechanical ventilation. Performs venous and arterial punctures. Obtains blood samples. Performs and manages regional anesthetic. Manages patient’s fluid, blood, electrolyte 

In [38]:
predict_jobs(model=model, df=df, text=example_data_engineer, topn=20)


Searching for matches for the following document: 
Data Engineer Job Responsibilities:  Develops and maintains scalable data pipelines and builds out new API integrations to support continuing increases in data volume and complexity. Collaborates with analytics and business teams to improve data models that feed business intelligence tools, increasing data accessibility and fostering data-driven decision making across the organization. Implements processes and systems to monitor data quality, ensuring production data is always accurate and available for key stakeholders and business processes that depend on it. Writes unit/integration tests, contributes to engineering wiki, and documents work. Performs data analysis required to troubleshoot data related issues and assist in the resolution of data issues. Works closely with a team of frontend and backend engineers, product managers, and analysts. Defines company data assets (data models), spark, sparkSQL, and hiveSQL jobs to populate d

## Make some fake resumes now...

In [43]:
resume_software_enginer = "Summary Enthusiastic software engineer with 4+ years experience participating in the complete product development lifecycle of successfully launched applications. Eager to join XYZ Inc. to deliver mission-critical technology and business solutions to Fortune 500 companies and some of the most recognized brands on the planet. In previous roles, reduced downtime by 15% and warranty costs by 25%; identified and resolved a process bottleneck that reduced coding efficiency by up to 30%. Experience Software Engineer  Cloud Nin9 August 2017–present  Built modern applications with JAVA, Spring, Spring Boot, SQL Server, No SQL. Developed microservices and Web Services (incl. REST/SOAP/WSDL/XML/SOA). Built on Pivotal Cloud Foundry Platform (Gradle, GitHub). Continuously integrated and deployed developed software. Updated the continuous integration/deployment scripts as necessary to improve continuous integration practices. Consulted with Product Manager to identify minimal viable product and decomposed feature set into small scoped user stories. Key achievement:  Ensured the uninterrupted flow of business-critical operations. On-time error analysis reduced downtime by 15% and costs of warranty by up to 25%.    Software Engineer Intern CodEX September 2016–May 2017 Supported CodEX software development and testing processes. Verified that software met requirements. Maintained existing applications. Collaborated on future projects and innovations. Continuously identified, measured, and improved processes. Key achievement: Identified and resolved a process bottleneck that reduced coding efficiency by up to 30%. Education B.Sc., Software Engineering  University of Washington Bothell 2013–2016 Heavy concentration in data structures, Java, SQL. Organized the university hackathon in two consecutive years. Built a mock customer service web app for a Senior Scholar project. Skills Adaptability Agile processes Excellent communication skills JAVA, Spring, Spring Boot, SQL Server, No SQL Microservices development Pivotal Cloud Foundry Platform (Gradle, GitHub) Status tracking tools (Jira and Rally) Strong collaboration skills Practical knowledge of SQL and database concepts Certifications Oracle Certified Associate (OCA) Java SE Programmer CIW Web Development Professional "
resume_marketing = "An organized and creative professional with proven marketing skills and a desire to learn more. Possess 6+ years of experience working for a diverse group of organizations and clients. Gained expertise in data analysis, publication marketing, and web-based advertising. Looking to utilize my existing skill set to increase company profitability. PROFESSIONAL EXPERIENCE BUSINESS CONSULTANT Claydon Heeley, Boston, MA  /  September 2016 – Present Assisted clients with business plan, purchase, branding, advertising, remodeling, and marketing, increasing monthly sales from $8,000 to $25,000 in ten months Increased followers on social media by 150% on average, resulting in $45,000 in increased sales via the Internet through a significant increase in brand awareness Initiated a series of marketing campaigns including email, print, digital, outdoor and social media analysis, Crafted concept and content for journal ads, direct mail campaigns and blogs ASSOCIATE PUBLISHER The Boston Herald, Boston, MA  /  June 2013 – August 2016 Spearheaded the development of the first media kit amalgamation for all company projects, increasing national sales by 8% Implemented efficiency and cost-saving programs that reworked the editorial, production, and sales processes for three magazines; increasing profitability by 10% on average Kept abreast with the latest publication marketing strategies and developed new social media campaigns to increase brand influence Discovered, negotiated, and secured new relationships with local organizations, resulting in the launch of five new publications whose profitability increased between 10 – 20% annually EDUCATION B.A. in Art & Design Watford Art School, Chicago, IL 2009-2013 SKILLS Adobe Creative Suite Google Analytics Webmaster Tools Social Media Marketing Stata / Atlas Asana / Jira"





In [44]:
predict_jobs(model=model, df=df, text=resume_software_enginer, topn=20)


Searching for matches for the following document: 
Summary Enthusiastic software engineer with 4+ years experience participating in the complete product development lifecycle of successfully launched applications. Eager to join XYZ Inc. to deliver mission-critical technology and business solutions to Fortune 500 companies and some of the most recognized brands on the planet. In previous roles, reduced downtime by 15% and warranty costs by 25%; identified and resolved a process bottleneck that reduced coding efficiency by up to 30%. Experience Software Engineer  Cloud Nin9 August 2017–present  Built modern applications with JAVA, Spring, Spring Boot, SQL Server, No SQL. Developed microservices and Web Services (incl. REST/SOAP/WSDL/XML/SOA). Built on Pivotal Cloud Foundry Platform (Gradle, GitHub). Continuously integrated and deployed developed software. Updated the continuous integration/deployment scripts as necessary to improve continuous integration practices. Consulted with Produc

In [45]:
predict_jobs(model=model, df=df, text=resume_marketing, topn=20)


Searching for matches for the following document: 
An organized and creative professional with proven marketing skills and a desire to learn more. Possess 6+ years of experience working for a diverse group of organizations and clients. Gained expertise in data analysis, publication marketing, and web-based advertising. Looking to utilize my existing skill set to increase company profitability. PROFESSIONAL EXPERIENCE BUSINESS CONSULTANT Claydon Heeley, Boston, MA  /  September 2016 – Present Assisted clients with business plan, purchase, branding, advertising, remodeling, and marketing, increasing monthly sales from $8,000 to $25,000 in ten months Increased followers on social media by 150% on average, resulting in $45,000 in increased sales via the Internet through a significant increase in brand awareness Initiated a series of marketing campaigns including email, print, digital, outdoor and social media analysis, Crafted concept and content for journal ads, direct mail campaigns and